## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [195]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [196]:
pwd

'D:\\AIHUB'

### Function

In [ ]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [197]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [198]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [199]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [200]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):

  text_file_name_list = [folder_corpus_type_name + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_name_list

In [201]:
def post_txt_file_name_list(corpus_list):
   
  post_corpus_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_list]

  post_corpus_list = sorted_list(post_corpus_list)

  return post_corpus_list

In [202]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}|\<.*?\>", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
            # ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외
            
                                    
            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)
                        
            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

### AIHUB 기계독해

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=89)

In [203]:
path_list = ['AIHUB_기계독해'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [204]:
the_number_of_file = len(file_name) 
print("The number of file:", the_number_of_file)
n = 1
print("The number of list element:", n)
file_nested_list = list(divide_source_file_list(file_name, n))

The number of file: 3
The number of list element: 1


In [205]:
text_file_name_list = txt_file_name_list(file_nested_list,
                                         "exploration/machine_reading_pro/AIHUB_machine_reading_")
the_numer_of_txt_file = len(text_file_name_list)
print("The number of txt file:", the_numer_of_txt_file)

The number of txt file: 3


In [206]:
def list_length_checker(source_file_nested_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    temp_nested_index = []
    
    for source_file_list in source_file_nested_list:   
        temp_index = []
        for i in range(len(source_file_list)):

            if type(source_file_list) == str:
              source_file = source_file_list

            elif type(source_file_list) != str:
              source_file = source_file_list[i]

            with open(source_file, 'r', encoding='utf-8') as one_json_file:
              one_json_sample = json.load(one_json_file)

            source_df = pd.DataFrame(one_json_sample['data'])
            source_dict = dict(source_df['paragraphs'].explode())
            source_json = pd.json_normalize(source_dict)
            source_list = list(source_json.filter(regex='context').values[0])

            the_number_of_txt_file = ((len(source_list) // batch_size) + 1)

            if len(source_list) >= 1000:
                print("File:", source_file)    
                print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                    "  ", "The number of txt file:", the_number_of_txt_file, "\n")
                the_number_of_txt_file_list.append(the_number_of_txt_file)
                the_number_of_total_txt_file  += the_number_of_txt_file
            else:
                the_number_of_total_txt_file  += 1
                the_number_of_txt_file_list.append(1)
                if i > 0 and the_number_of_file % i == 0:
                    temp_index.append(i)
                    try:
                        if i == temp_nested_index[0][0] and len(temp_nested_index) <= 1:
                            print("[For Example]")
                            print("This is not subject of batch. It's small source list.")                            
                            print("File:", source_file)
                            print("Length of Source List:", len(source_list), 
                                    "  ", "The number of txt file:", 1, "\n") 
                    except:
                        pass

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [207]:
def make_corpus_txt_with_batch_list(source_file_nested_list,
                                    text_file_name_list,
                                    batch_size, the_number_of_txt_file_list):

  print("[Size]")
  print("The number of preprocessing corpus: " + str(sum(the_number_of_txt_file_list)))
  print("\n[Order]")
  num = 0
  for i in range(len(source_file_nested_list)):
    source_file_list = source_file_nested_list[i]
    
    for j in range(len(source_file_list)):
      
      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) != str:
        source_file = source_file_list[j]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      source_df = pd.DataFrame(one_json_sample['data'])
      source_dict = dict(source_df['paragraphs'].explode())
      source_json = pd.json_normalize(source_dict)
      source_list = list(source_json.filter(regex='context').values[0])

      n = batch_size
      source_batch_list = list(divide_source_file_list(source_list, n))
        
      for source_list in source_batch_list:
          num += 1
          print(str(num), end=" ")
          
          with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
              fp.write("\n".join(source_list))   

In [208]:
batch_size = 1000
the_number_of_txt_file, the_number_of_txt_file_list = list_length_checker(file_nested_list, batch_size)

File: AIHUB_기계독해\ko_nia_clue0529_squad_all.json
Index: 0    Length of Source List: 34500    The number of txt file: 35 

File: AIHUB_기계독해\ko_nia_noanswer_squad_all.json
Index: 0    Length of Source List: 20030    The number of txt file: 21 

File: AIHUB_기계독해\ko_nia_normal_squad_all.json
Index: 0    Length of Source List: 47314    The number of txt file: 48 

Batch Size: 1000
The number of txt file: 104


In [209]:
batch_size = 1000
make_corpus_txt_with_batch_list(file_nested_list, text_file_name_list,
                batch_size, the_number_of_txt_file_list)

[Size]
The numnber of preprocessing corpus: 104

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 

In [210]:
pro_total_corpus_list = glob("AIHUB_corpus/exploration/machine_reading_pro/AIHUB_machine_reading_" +"*.txt")

In [211]:
len(pro_total_corpus_list)

104

In [212]:
post_total_corpus_list = post_txt_file_name_list(pro_total_corpus_list)

In [213]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
        
for line in line_list:
    print(line, end="\n\n")

재개발을 앞두고 먼저 이사 간 빈 집의 수도가 동파해 남아있는 주민들이 불편을 겪고 있다. 경기도 안양시 호계동 재개발단지의 한 건물 외벽은 거대한 고드름으로 덮여 있고 주민들은 하루 종일 새는 물을 퍼내기 바빴다.    이 지역 내 빈집 등 30여 곳에서 수도 동파가 발생했지만 이를 관리할 사람들이 없다. 미처 이사를 가지 못한 거주민들은 새는 물을 대야로 바치거나 습기가 가득 찬 마루 바닥을 온풍기로 말리며 힘겨운 겨울을 나고 있다.    한파가 몰아친 4일 거주민 정모씨(79)의 집은 전쟁터를 방불케 한다. 현관등이 있는 천정에서 물이 새면서 전기 누전으로 연기가 나고, 고드름을 제거하다 실수로 베란다 창문마저 깨졌다. 정씨는 &quot;이 엄동설한에 어디로 나가야 하는 지 막막하다&quot;며 &quot;여기 저기 물이 새 집 안에서도 신발을 신고 다녀야 한다&quot;고 하소연 했다.   지하에 사는 이모씨(60) 역시 바닥에 차는 물을 퍼내며 하루를 보낸다. 이씨는 &quot;30년간 살면서 물이 새본 적이 없는데, 마룻바닥이 그새 변색됐다&quot;며 &quot;재개발 때문에 이렇게 됐지만 마땅한 대책이 없다&quot;고 말했다.    재개발구역 내 아직 이주하지 못한 주민들은 남은 겨울을 보내기가 두렵기만 하다. 안양 호계동의 재개발단지에는 아직 700세대가 남아 있다.  안양 = 최현규 기자 frosted@kmib.co.kr



In [214]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

재개발을 앞두고 먼저 이사 간 빈 집의 수도가 동파해 남아있는 주민들이 불편을 겪고 있다.

경기도 안양시 호계동 재개발단지의 한 건물 외벽은 거대한 고드름으로 덮여 있고 주민들은 하루 종일 새는 물을 퍼내기 바빴다.

이 지역 내 빈집 등 30여 곳에서 수도 동파가 발생했지만 이를 관리할 사람들이 없다.

미처 이사를 가지 못한 거주민들은 새는 물을 대야로 바치거나 습기가 가득 찬 마루 바닥을 온풍기로 말리며 힘겨운 겨울을 나고 있다.

한파가 몰아친 4일 거주민 정모씨(79)의 집은 전쟁터를 방불케 한다.

현관등이 있는 천정에서 물이 새면서 전기 누전으로 연기가 나고, 고드름을 제거하다 실수로 베란다 창문마저 깨졌다.

정씨는  quot 이 엄동설한에 어디로 나가야 하는 지 막막하다 quot 며  quot 여기 저기 물이 새 집 안에서도 신발을 신고 다녀야 한다 quot 고 하소연 했다.

지하에 사는 이모씨(60) 역시 바닥에 차는 물을 퍼내며 하루를 보낸다.

이씨는  quot 30년간 살면서 물이 새본 적이 없는데, 마룻바닥이 그새 변색됐다 quot 며  quot 재개발 때문에 이렇게 됐지만 마땅한 대책이 없다 quot 고 말했다.

재개발구역 내 아직 이주하지 못한 주민들은 남은 겨울을 보내기가 두렵기만 하다.

안양 호계동의 재개발단지에는 아직 700세대가 남아 있다.



In [215]:
ray.init()

@ray.remote

def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}|\<.*?\>", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
            # ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외
            
                                    
            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)
                        
            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

2023-05-26 14:57:45,327	INFO worker.py:1625 -- Started a local Ray instance.


In [222]:
print("[Size]")
print("The number of preprocessing corpus: " + str(len(pro_total_corpus_list)))
print("\n[Order]")
num = 0
process_num = 10    
for pro, post in zip(pro_total_corpus_list, post_total_corpus_list):
    
    with open(pro, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines() 
        nested_lines_num = len(lines) // process_num
        for i in range(nested_lines_num - 1):
            start_line = process_num * i
            end_line = process_num * (i+1)
            futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
            results = ray.get(futures)
            if i == nested_lines_num - 2:
                futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                results = ray.get(futures)
        sentence_list = list(chain.from_iterable(results))

    num += 1
    print(str(num), end=" ")  
        
    with open(post, 'a', encoding='utf-8') as fp:
        fp.write("\n".join(sentence_list))

[Size]
The numnber of preprocessing corpus: 104

[Order]
1 

KeyboardInterrupt: 

In [223]:
ray.shutdown()

In [ ]:
corpus_list = glob("AIHUB_corpus/exploration/machine_reading_post/AIHUB_machine_reading_" +"*.txt")

In [ ]:
with open('AIHUB_corpus/duplicate/AIHUB_machine_reading.txt', 'w') as f:
    for corpus in corpus_list:
        with open(corpus) as text:
            for line in text:
                f.write(line)

In [ ]:
with open('AIHUB_corpus/AIHUB_machine_reading.txt', 'w', encoding='utf-8') as f1:
    with open('AIHUB_corpus/duplicate/AIHUB_machine_reading.txt', encoding='utf-8') as f2:
        lines = f2.read().splitlines()
        single_sentence_dict = dict.fromkeys(lines)
        single_sentence_list = list(single_sentence_dict)
        f1.write("\n".join(single_sentence_list))